In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from boto.s3.connection import S3Connection
from boto.s3.key import Key
from boto import s3
import boto3
from boto import boto
import re
import ssl
import pandas as pd
import numpy as np
from configparser import ConfigParser
import json
import pickle

In [3]:
import configparser
import os.path
from os import path
from importlib import reload


creds_path_ar = ["../../credentials.ini","credentials.colab.ini"]

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        AWS_ACCESS_KEY = config_parser['MAIN']["AWS_ACCESS_KEY"]
        AWS_ACCESS_SECRET_KEY = config_parser['MAIN']["AWS_ACCESS_SECRET_KEY"]
        PATH_ROOT = config_parser['MAIN']['PATH_ROOT']
        PATH_DATA = config_parser['MAIN']['PATH_DATA']
        break
BUCKET='sota-mafat'


In [4]:
cd {PATH_ROOT}

/home/ubuntu/sota-mafat-radar


In [5]:
def pull_alldata_s3(AWS_ACCESS_KEY=AWS_ACCESS_KEY,AWS_ACCESS_SECRET_KEY=AWS_ACCESS_SECRET_KEY,bucket=BUCKET,PATH_ROOT = PATH_ROOT):
    conn = S3Connection(AWS_ACCESS_KEY, AWS_ACCESS_SECRET_KEY)
    conn.auth_region_name = 'us-east-1.amazonaws.com'
    mybucket = conn.get_bucket(bucket)

    for key_name in mybucket.list():
        if (".csv" in str(key_name)) or (".pkl" in str(key_name)):
            key = mybucket.get_key(key_name.key)
            key.get_contents_to_filename(PATH_ROOT + '/data/' + key_name.name)

In [6]:
pull_alldata_s3()